## Visit Seoul Net

In [2]:
import json
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [3]:
base_url = 'https://korean.visitseoul.net'
url = f'{base_url}/restaurants?curPage=1'
result = requests.get(url)
soup = BeautifulSoup(result.text, 'html.parser')

- 한개를 선택해서 데이터 추출

In [5]:
lis = soup.select('li.item')
len(lis)

8

In [6]:
li = lis[0]
href = li.select_one('a')['href']
href

'/restaurants/통인시장/KOP000281'

In [7]:
img_src = li.select_one('.thumb > img')['src']
img_src

'/data/MEDIA/20240516/0516153423441-1963d7e37b9340c4bf45be6ef00a1580.jpg'

In [8]:
sub_url = f'{base_url}{href}'
sub_res = requests.get(sub_url)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [9]:
name = sub_soup.select_one('.h3.textcenter').text.strip()
name

'통인시장'

In [10]:
intro = sub_soup.select_one('.text-area').get_text().strip().replace(u'\xa0', u' ')
intro

'세종마을은 추사 김정희 등 조선시대 예능인들이 모이는 중심지였으며 근대에도 이상 등 문인들이 활동하던 중심지였다. 일제강점기 일본인들을 위해 만들어진 공설시장이 모태이나 6.25이후 인구증가로 시장의 필요성이 높아져 현재처럼 물건을 사고파는 시장의 형태를 갖추게 되었다.    통인시장은 상권이 활성화된 곳으로 다른 전통시장의 벤치마킹 사례로 유명하며 특히 통인시장에서 자체 제작한 엽전은 통인시장만의 즐길 거리다. 엽전으로 환전함과 동시에 검은색 플라스틱 식판도 받게 되는데 이때부터 식판을 들고 다니면서 시장 곳곳을 다니며 음식을 먹을 수 있다. 단, 엽전은 가맹점에서만 사용 가능하다.'

In [11]:
dls = sub_soup.select('.detail-map-infor.first.border > dl')
len(dls)

7

In [12]:
info_dict = {}
for dl in dls:
    dt = dl.select_one('dt').text.strip()
    dd = dl.select_one('dd').text.strip().replace(u'\xa0', u' ').replace('\r', '\n')
    if dd == '웹사이트 보기':
        dd = dl.select_one('a')['href']
    info_dict[dt] = dd
info_dict

{'전화번호': '02-722-0911',
 '웹사이트': 'https://tonginmarket.modoo.at/',
 '홈페이지 언어': '한국어',
 '이용시간': '통인시장 영업시간 : 07:00 ~ 21:00\n※ 상점에 따라 상이하며, 정확한 개점, 폐점 시간은 없음\n 도시락 카페 영업시간 화~금 11:00 ~ 16:00, 토~일 11:00 ~ 17:00 \n※ 도시락 카페 엽전 판매는 평일 11:00 ~ 15:00, 주말 11:00 ~ 16:00까지',
 '휴무일': '통인시장 휴무일 : 매월 셋째주 일요일\n도시락 카페 휴무일 : 매주 월요일, 매월 셋째주 일요일',
 '주소': '03036  서울 종로구 자하문로15길 18',
 '교통 정보': '3호선 경복궁역 2번 출구에서 약 515m (도보 8분)'}

In [13]:
inner_divs = sub_soup.select('.trip-lst > li > div.inner')
len(inner_divs)

5

In [14]:
reviews = []
for inner_div in inner_divs:
    review = inner_div.select_one('dd').text.strip()
    score = inner_div.select_one('div.group-01 > img.trip-02')['alt'].split(':')[1]
    row = {'review': review, 'score': score}
    reviews.append(row)
reviews

[{'review': '생각보다 짧고 작은 시장입니다. 그러나 엽전은 정말 색다르고, 통인시장만의 색깔을 갖게 하는 좋은 아이디어인것 같습니다.\n 시장분들도 판매하는것 좋지만 차가운 음식은 전자렌지 돌리라는 안내좀 해주셨으면ㅠㅠ',
  'score': '4'},
 {'review': '시장에 장을 보러 간다기보다는 분위기를 느끼고 길거리음식들 사먹는 재미로 들러보면 좋을 것 같다\n물건들을 사는 방식이 옛날 조선시대 스타일이어서 아이들이 함께가도 재밌을것 같아요',
  'score': '3'},
 {'review': '북촌들려 경복궁 관람 후 서촌으로 넘어가서 통인시장마무리, 이 코스 나쁘지 않다. 통인시장은 엽전을 사고, 다니면서 원하는 음식을 도시락 용기에 담는다. 서서 먹어도 되지만 2층에 준비된 공간도 있다. 음식맛 최고인데 겨울에는 음식을 만드는 곳 앞에서 바로 먹는것을 추천한다. 담아서 다니다 보면 일찍 식어버려서 맛이 떨어진다. 주말에는 너무 복잡하고...',
  'score': '4'},
 {'review': '엽전을 이용한 도시락을 먹을 수 있어서 색다른 추억을 만들 수 있습니다. 기름떡볶이, 떡갈비 등 다양한 음식을 즐길 수 있어요. 다만 대체적으로 기름지거나 튀긴 음식이 많아요',
  'score': '3'},
 {'review': '종로 쪽 좋아해서 자주 다니는데, 주로 북촌만 가다가 서촌쪽에서 이곳에 처음 갔을 때 떡갈비의 맛에 반해버렸습니다. 그래서 저거 먹고싶어서라도 가요. 떡볶이떡같은 떡에 고기를 입혀서 구워주시는 느낌이에요. 아 물론 다른 가게들도 맛있습니다. 도시락카페 좋은 것 같아요',
  'score': '4'}]

- 한 페이지 내에 있는 정보 추출

In [15]:
lines = []
for li in lis:
    href = li.select_one('a')['href']
    img_src = li.select_one('.thumb > img')['src']
    sub_url = f'{base_url}{href}'
    sub_res = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

    name = sub_soup.select_one('.h3.textcenter').text.strip()
    intro = sub_soup.select_one('.text-area').get_text().strip().replace(u'\xa0', u' ')
    dls = sub_soup.select('.detail-map-infor.first.border > dl')
    info_dict = {}
    for dl in dls:
        dt = dl.select_one('dt').text.strip()
        dd = dl.select_one('dd').text.strip().replace(u'\xa0', u' ').replace('\r', '\n')
        if dd == '웹사이트 보기':
            dd = dl.select_one('a')['href']
        info_dict[dt] = dd

    inner_divs = sub_soup.select('.trip-lst > li > div.inner')
    reviews = []
    for inner_div in inner_divs:
        review = inner_div.select_one('dd').text.strip()
        score = inner_div.select_one('div.group-01 > img.trip-02')['alt'].split(':')[1]
        row = {'review': review, 'score': score}
        reviews.append(row)

    lines.append({'상호': name, 'img_src': f'{base_url}{img_src}', '설명': intro, '업소정보': json.dumps(info_dict), '리뷰': json.dumps(reviews)})

In [16]:
df = pd.DataFrame(lines)
df.head()

,상호,img_src,설명,업소정보,리뷰
0,통인시장,https://korean.visitseoul.net/data/MEDIA/20240...,세종마을은 추사 김정희 등 조선시대 예능인들이 모이는 중심지였으며 근대에도 이상 등...,"{""\uc804\ud654\ubc88\ud638"": ""02-722-0911"", ""\...","[{""review"": ""\uc0dd\uac01\ubcf4\ub2e4 \uc9e7\u..."
1,두레한식당,https://korean.visitseoul.net/data/POST/201612...,"60여 년 전 경남 밀양에서 개업한 이래, 인사동에서 2대에 걸쳐 명맥을 이어온 유...","{""\uc804\ud654\ubc88\ud638"": ""02-3213-2638"", ""...",[]
2,카몽(KAMONG),https://korean.visitseoul.net/data/POST/201612...,엑소의 멤버 ‘카이가 꿈을 꾸다’라는 뜻을 가진 카페 카몽은 카이의 누나가 운영하고...,"{""\uc804\ud654\ubc88\ud638"": ""0507-1498-5951"",...",[]
3,놀부유황오리진흙구이(잠실점),https://korean.visitseoul.net/data/POST/201601...,보약을 다리는 정성으로 구워내는 유황오리. 유황오리 진흙구이는 황토진흙으로 만든 토...,"{""\uc804\ud654\ubc88\ud638"": ""0507-1383-5292"",...","[{""review"": ""\uc624\ub9ac\uad6c\uc774\ub294 \u..."
4,클럽 NB2,https://korean.visitseoul.net/data/MEDIA/20241...,강남역 거리 및 홍대 클럽 거리에 위치한 대형 힙합댄스 클럽이다. ’99년 말 홍대...,"{""\uc804\ud654\ubc88\ud638"": ""0507-1323-8213"",...","[{""review"": ""\uc194\uc9c1\ud788 \uc881\uc9c0\u..."


- 모든 페이지에 대해 적용

In [17]:
lines = []
for page in tqdm(range(1, 122)):
    url = f'{base_url}/restaurants?curPage={page}'
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    lis = soup.select('li.item')

    for li in lis:
        href = li.select_one('a')['href']
        img_src = li.select_one('.thumb > img')['src']
        sub_url = f'{base_url}{href}'
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

        name = sub_soup.select_one('.h3.textcenter').text.strip()
        intro = sub_soup.select_one('.text-area').get_text().strip().replace(u'\xa0', u' ')
        dls = sub_soup.select('.detail-map-infor.first.border > dl')
        info_dict = {}
        for dl in dls:
            dt = dl.select_one('dt').text.strip()
            dd = dl.select_one('dd').text.strip().replace(u'\xa0', u' ').replace('\r', '\n')
            if dd == '웹사이트 보기':
                dd = dl.select_one('a')['href']
            info_dict[dt] = dd

        inner_divs = sub_soup.select('.trip-lst > li > div.inner')
        reviews = []
        for inner_div in inner_divs:
            review = inner_div.select_one('dd').text.strip()
            score = inner_div.select_one('div.group-01 > img.trip-02')['alt'].split(':')[1]
            row = {'review': review, 'score': score}
            reviews.append(row)

        lines.append({'상호': name, 'img_src': f'{base_url}{img_src}', '설명': intro, '업소정보': json.dumps(info_dict), '리뷰': json.dumps(reviews)})

100%|██████████| 121/121 [13:39<00:00,  6.77s/it]


In [18]:
df = pd.DataFrame(lines)
df.to_csv('서울맛집3.csv', index=False)